# Bayesian Optimization on Keras

### MNIST training on Keras with Bayesian optimization
* This notebook runs MNIST training on Keras using Bayesian optimization to find the best hyper parameters.
* The MNIST model here is just a simple one with one input layer, one hidden layer and one output layer, without convolution.
* Hyperparameters of the model include the followings:
* - output shape of the first layer
* - dropout rate of the first layer
* - output shape of the second layer
* - dropout rate of the second layer
* - use batchnormalization or not
* - batch size
* - number of epochs
* - validation rate
* I used GPy and GPyOpt to run Bayesian optimization.

#### Import libraries

In [13]:
import GPy, GPyOpt
import numpy as np
import pandas as pds
import random
from keras.layers import Activation, Dropout, BatchNormalization, Dense
from keras.models import Sequential
from keras.datasets import mnist
from keras.metrics import categorical_crossentropy
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

#### Define MNIST model
* includes data loading function, training function, fit function and evaluation function 

In [14]:
# MNIST class
class MNIST():
    def __init__(self,
                 l1_out=512, 
                 l2_out=512, 
                 l1_drop=0.2, 
                 l2_drop=0.2, 
                 bn1=0,
                 bn2=0,
                 batch_size=100, 
                 epochs=10, 
                 validation_split=0.1):
        self.l1_out = l1_out
        self.l2_out = l2_out
        self.l1_drop = l1_drop
        self.l2_drop = l2_drop
        self.bn1 = bn1
        self.bn2 = bn2
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.__x_train, self.__x_test, self.__y_train, self.__y_test = self.mnist_data()
        self.__model = self.mnist_model()
        
    # load mnist data from keras dataset
    def mnist_data(self):
        (X_train, y_train), (X_test, y_test) = mnist.load_data()
        X_train = X_train.reshape(60000, 784)
        X_test = X_test.reshape(10000, 784)

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255
        X_test /= 255

        Y_train = np_utils.to_categorical(y_train, 10)
        Y_test = np_utils.to_categorical(y_test, 10)
        return X_train, X_test, Y_train, Y_test
    
    # mnist model
    def mnist_model(self):
        model = Sequential()
        model.add(Dense(self.l1_out, input_shape=(784,)))
        if self.bn1 == 0:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(self.l1_drop))
        model.add(Dense(self.l2_out))
        if self.bn2 == 0:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(self.l2_drop))
        model.add(Dense(10))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

        return model
    
    # fit mnist model
    def mnist_fit(self):
        early_stopping = EarlyStopping(patience=0, verbose=1)
        
        self.__model.fit(self.__x_train, self.__y_train,
                       batch_size=self.batch_size,
                       epochs=self.epochs,
                       verbose=0,
                       validation_split=self.validation_split,
                       callbacks=[early_stopping])
    
    # evaluate mnist model
    def mnist_evaluate(self):
        self.mnist_fit()
        
        evaluation = self.__model.evaluate(self.__x_test, self.__y_test, batch_size=self.batch_size, verbose=0)
        return evaluation

#### Runner function for the MNIST model

In [15]:
# function to run mnist class
def run_mnist(l1_out=512, l2_out=512, 
              l1_drop=0.2, l2_drop=0.2, 
              bn1=0, bn2=0,
              batch_size=100, epochs=10, validation_split=0.1):
    
    _mnist = MNIST()
    mnist_evaluation = _mnist.mnist_evaluate()
    return mnist_evaluation

## Bayesian Optimization
#### bounds for hyper parameters

In [16]:
# bounds for hyper-parameters in mnist model
# the bounds dict should be in order of continuous type and then discrete type
bounds = [{'name': 'validation_split', 'type': 'continuous',  'domain': (0.0, 0.3)},
          {'name': 'l1_drop',          'type': 'continuous',  'domain': (0.0, 0.3)},
          {'name': 'l2_drop',          'type': 'continuous',  'domain': (0.0, 0.3)},
          {'name': 'l1_out',           'type': 'discrete',    'domain': (64, 128, 256, 512, 1024)},
          {'name': 'l2_out',           'type': 'discrete',    'domain': (64, 128, 256, 512, 1024)},
          {'name': 'bn1',              'type': 'discrete',    'domain': (0, 1)},
          {'name': 'bn2',              'type': 'discrete',    'domain': (0, 1)},
          {'name': 'batch_size',       'type': 'discrete',    'domain': (10, 100, 500)},
          {'name': 'epochs',           'type': 'discrete',    'domain': (5, 10, 20)}]

#### Bayesian Optimization

In [17]:
# function to optimize mnist model
def f(x):
    print(x)
    evaluation = run_mnist(
        l1_drop = int(x[:,1]), 
        l2_drop = int(x[:,2]), 
        l1_out = float(x[:,3]),
        l2_out = float(x[:,4]), 
        bn1 = int(x[:,5]),
        bn2 = int(x[:,6]),
        batch_size = int(x[:,7]), 
        epochs = int(x[:,8]), 
        validation_split = float(x[:,0]))
    print("loss:{0} \t\t accuracy:{1}".format(evaluation[0], evaluation[1]))
    return evaluation[0]

#### Optimizer instance

In [18]:
# optimizer
opt_mnist = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds)

[[  5.39697936e-02   1.94312100e-02   2.38992496e-01   5.12000000e+02
    1.28000000e+02   1.00000000e+00   0.00000000e+00   1.00000000e+01
    1.00000000e+01]]
Epoch 00005: early stopping
loss:0.06506255014410271 		 accuracy:0.980400008559227
[[  1.09812226e-01   2.27021432e-01   2.96083137e-01   1.28000000e+02
    6.40000000e+01   0.00000000e+00   0.00000000e+00   5.00000000e+02
    1.00000000e+01]]
Epoch 00003: early stopping
loss:0.07566350179942674 		 accuracy:0.9779000067710877
[[  2.23116113e-01   3.03583663e-02   5.15287406e-02   1.28000000e+02
    5.12000000e+02   0.00000000e+00   1.00000000e+00   5.00000000e+02
    2.00000000e+01]]
Epoch 00002: early stopping
loss:0.07970559070403396 		 accuracy:0.9754000079631805
[[  2.40348118e-01   2.22332835e-02   7.15477484e-02   5.12000000e+02
    6.40000000e+01   0.00000000e+00   1.00000000e+00   1.00000000e+01
    2.00000000e+01]]
Epoch 00004: early stopping
loss:0.07282492852740688 		 accuracy:0.9772000026702881
[[  2.24195230e-01   

#### Running optimization

In [19]:
# optimize mnist model
opt_mnist.run_optimization(max_iter=15)

[[  3.00000000e-01   3.00000000e-01   0.00000000e+00   5.12000000e+02
    1.28000000e+02   1.00000000e+00   0.00000000e+00   1.00000000e+01
    1.00000000e+01]]
Epoch 00001: early stopping
loss:0.0909516843734309 		 accuracy:0.9704000043869019
[[  5.39752583e-02   1.94382634e-02   2.38996765e-01   5.12000000e+02
    1.28000000e+02   1.00000000e+00   0.00000000e+00   1.00000000e+01
    1.00000000e+01]]
Epoch 00002: early stopping
loss:0.08541888958308846 		 accuracy:0.972100003361702
[[  2.34766818e-01   2.69630386e-01   8.39893790e-02   5.12000000e+02
    1.28000000e+02   1.00000000e+00   0.00000000e+00   5.00000000e+02
    2.00000000e+01]]
Epoch 00004: early stopping
loss:0.07459711219664314 		 accuracy:0.9762000066041946
[[  2.34766818e-01   2.69630386e-01   8.39893790e-02   5.12000000e+02
    1.28000000e+02   1.00000000e+00   0.00000000e+00   5.00000000e+02
    2.00000000e+01]]
Epoch 00003: early stopping
loss:0.07821810089924838 		 accuracy:0.975700004696846


#### The output

In [20]:
optbounds = {
    'validation_split' : opt_mnist.x_opt[0],
    'l1_drop' : opt_mnist.x_opt[1],
    'l2_drop' : opt_mnist.x_opt[2],
    'l1_out' : opt_mnist.x_opt[3],
    'l2_out' : opt_mnist.x_opt[4],
    'bn1' : opt_mnist.x_opt[5],
    'bn2' : opt_mnist.x_opt[6],
    'batch_size' : opt_mnist.x_opt[7],
    'epochs' : opt_mnist.x_opt[8],
}

In [21]:
# print optimized mnist model
print("optimized parameters: {0}".format(optbounds))
print("optimized loss: {0}".format(opt_mnist.fx_opt))

optimized parameters: {'l1_drop': 0.019431209972365925, 'batch_size': 10.0, 'epochs': 10.0, 'validation_split': 0.053969793626947433, 'l2_drop': 0.23899249584771631, 'bn2': 0.0, 'l1_out': 512.0, 'bn1': 1.0, 'l2_out': 128.0}
optimized loss: [ 0.06506255]
